## Fine-Tuning ModernBERT: Exploring a Lightweight Approach to Prompt Guardrails (Notebook)

In [ ]:
%pip install transformers accelerate flash-attn --no-build-isolation --quiet | tail -n 1


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### Dataset Preparation

In [ ]:
from datasets import Dataset, DatasetDict, Split

ds = Dataset.from_json('train.json').shuffle(seed=42)
ds = ds.select(range(15000))

In [ ]:
ds = ds.train_test_split(test_size=0.25)

ds = DatasetDict({
    "train": ds['train'],
    "test": ds['test']
})

Dataset({
    features: ['prompt', 'label', 'source'],
    num_rows: 76735
})

In [ ]:
ds_train = ds["train"]

### 3. Tokenization

Tokenization is a foundational process to transform text into a format that models can understand. It works by splitting an input string into smaller units called tokens and mapping each token to a unique numerical ID from the model's vocabulary. Depending on the tokenization strategy, these tokens might represent whole words, subwords, or individual characters. The numerical IDs act as indexes into the token embeddings, where each token is represented as a dense vector capturing its semantic properties.

ModernBERT uses a subword tokenization method based on a modified version of BPE, OLMo tokenizer that can handle out-of-vocabulary words by breaking an input into subword units from a 50,368 vocabulary (multiple of 64). In this section, we use the AutoTokenizer from the Transformers library to tokenize our prompt sentences. The tokenizer is initialized with the same checkpoint as our model (ModernBERT-base) to ensure compatibility.

In [ ]:
from transformers import AutoTokenizer

checkpoint = "answerdotai/ModernBERT-base"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize(batch):
    return tokenizer(batch['prompt'], truncation=True)

The `tokenize` function processeses the prompt sentences, applying truncation to fit `ModernBERT` maximum sequence length of 8192 tokens. The map function is then used to apply this tokenization to our entire dataset efficiently.

In [ ]:
t_ds = ds.map(tokenize, batched=True)

#### 3.1 The [CLS] and [SEP ]special tokens

In practice, models like ModernBERT are designed with specific special tokens in mind, such as [CLS] and [SEP] to guide the model's understanding of input sequences.

The [CLS] stands for Classification and is placed at the beginning of every input sequence. As the input passes through the model's encoder layers, this token will progressively accumulate contextual information from the entire sequence (through the self-attention mechanisms). Its final-layer representation will be then passed into our classification head (a feed-forward neural network).

The [SEP] token stands for Separator and is used to separate different segments of text within an input sequence. This token is particular relevant for tasks like next sentence prediction, where the model needs to determine if two sentences are related. The [SEP] token helps the model understand which tokens belong to which sentence.

In [ ]:
ds_train[42]

In [ ]:
tokenize(ds_train[42])

In [ ]:
tokenizer.decode(t_ds["train"][42]["input_ids"])

#### 3.2 Data Collation

Dynamic padding is an efficient technique used to handle variable-length sequences within a batch. Instead of padding all sequences to a fixed maximum length, which can waste computational resources, dynamic padding adds padding only up to the length of the longest sequence in each batch. This approach optimizes memory usage and computation time.

In our ModernBERT fine-tuning process, we willll use the DataCollatorWithPadding class from the Transformers library, which automatically pads the inputs in each batch to the maximum length in that specific batch.

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

### 4.  Fine-tune & Evaluate ModernBERT with Hugging Face

In [ ]:
from transformers import AutoModelForSequenceClassification

In [ ]:
labels = ['safe', 'unsafe']
num_labels = len(labels)
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label

In [ ]:
checkpoint = "answerdotai/ModernBERT-base"
model = AutoModelForSequenceClassification.from_pretrained(
    checkpoint, num_labels=num_labels, label2id=label2id, id2label=id2label, trust_remote_code=True
)

In [1]:
import numpy as np
from sklearn.metrics import accuracy_score, f1_score
 
# Metric helper method
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)
    f1 = f1_score(labels, predictions, labels=labels, pos_label=1, average="weighted")
    accuracy = accuracy_score(labels, predictions)

    return {"f1": float(f1) if f1 == 1 else f1, "accuracy": accuracy}

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir= "modernbert-promptguard",
    per_device_train_batch_size=32,
    per_device_eval_batch_size=16,
    learning_rate=5e-5,
	num_train_epochs=3,
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=3,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    optim="adamw_torch_fused",
    report_to="none",
    bf16=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=t_ds["train"],
    eval_dataset=t_ds["test"],
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

In [ ]:
trainer.save_model("md-guard-large")